#**Drive mount**

In [ ]:
from google.colab import drive
import shutil
drive.mount('/content/drive')

Mounted at /content/drive


#**Imports**

In [ ]:
import torchvision.transforms.functional as TF
import sklearn.metrics
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.models as models
from torchvision.models.resnet import resnet50
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
import matplotlib.pyplot as plt
from torchvision.io import read_image
from google.colab.patches import cv2_imshow
from torchsummary import summary
import torchvision
import numpy as np
import random
import os
import matplotlib.pyplot as plt
import cv2
from torchvision import datasets
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn import metrics
import xgboost as xgb

#**Paths**

In [ ]:
data_path = '/content/drive/MyDrive/Tirocinio/Datasets/SocialMemoryImgs/Wavelet_Abs_Plt_Jet'
train_path = '/content/drive/MyDrive/Tirocinio/Datasets/SocialMemoryImgs/Wavelet_Abs_Plt_Jet/sub50/train'
test_path = '/content/drive/MyDrive/Tirocinio/Datasets/SocialMemoryImgs/Wavelet_Abs_Plt_Jet/sub50/test'
val_path = '/content/drive/MyDrive/Tirocinio/Datasets/SocialMemoryImgs/Wavelet_Abs_Plt_Jet/sub50/val'

#**Carico il dataset in train, path e val trasformandolo direttamente con il trasformatore**

In [ ]:
def ttv_dataset(train_path, test_path, val_path, train_transforms, test_transforms, val_transforms):

    train_dataset = torchvision.datasets.ImageFolder(root = train_path, transform = train_transforms)
    test_dataset = torchvision.datasets.ImageFolder(root = test_path, transform = test_transforms)
    val_dataset = torchvision.datasets.ImageFolder(root = val_path, transform = val_transforms)

    return train_dataset, test_dataset, val_dataset


#**Suddivido in dataloader per velocizzare l'allenamento**

In [ ]:
def ttv_loader(train_dataset, test_dataset, val_dataset):

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 1, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 1, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 1, shuffle=True)

    return train_loader, test_loader, val_loader

#**Calcolo della mean e std per la normalizzazione valido sia per Grayscale che RGB, basta modificare il trasformatore**

In [ ]:
def mean_and_std(loader):

  channels_sum, channels_squared_sum, num_batches = 0, 0, 0

  for images, _ in loader:
    channels_sum += torch.mean(images, dim=[0,2,3])
    channels_squared_sum += torch.mean(images**2, dim=[0,2,3])
    num_batches += 1

  mean = channels_sum/num_batches
  std = (channels_squared_sum/num_batches - mean**2)**0.5

  return mean, std

In [ ]:
def set_device():
  if torch.cuda.is_available():
    dev = "cuda:0"
  else:
    dev = "cpu"
  return torch.device(dev)

In [ ]:
model = models.resnet50(pretrained=True)
model.fc = nn.Identity()
device = set_device()
model = model.to(device)

In [ ]:
def pretraining(train_loader, test_loader, val_loader):

    batch_t_range = range(0,40)
    batch_te_range = range(0,10)
    images_total_t = []
    images_total_te = []
    images_total_v = []
    labels_total_t = []
    labels_total_te = []
    labels_total_v = []


    for batch_train in train_loader:
      images_t, labels_t = batch_train
      images_t = images_t.to(device)
      labels_t = labels_t.to(device)
      labels_t = labels_t.item()
      labels_total_t.append(labels_t)
      features_t = model(images_t)
      features_t = features_t.cpu().detach().numpy()
      images_total_t.append(features_t)

    images_total_t = np.squeeze(images_total_t)


    for batch_test in test_loader:
      images_te, labels_te = batch_test
      images_te = images_te.to(device)
      labels_te = labels_te.to(device)
      labels_te = labels_te.item()
      labels_total_te.append(labels_te)
      features_te = model(images_te)
      features_te = features_te.cpu().detach().numpy()
      images_total_te.append(features_te)

    images_total_te = np.squeeze(images_total_te)

    for batch_val in val_loader:
      images_v, labels_v = batch_val
      images_v = images_v.to(device)
      labels_v = labels_v.to(device)
      labels_v = labels_v.item()
      labels_total_v.append(labels_v)
      features_v = model(images_v)
      features_v = features_v.cpu().detach().numpy()
      images_total_v.append(features_v)

    images_total_v = np.squeeze(images_total_v)

    train_data_flatt = images_total_t
    test_data_flatt = images_total_te
    #test_data_flatt = list(np.concatenate((images_total_te,images_total_v), 0))
    train_labels = labels_total_t
    test_labels = labels_total_te
    #test_labels = list(np.concatenate((labels_total_te,labels_total_v), 0))

    return train_data_flatt, test_data_flatt, train_labels, test_labels

In [ ]:
def ml_classification(train_data_flatt, train_labels, test_data_flatt):
    # --SVM--
    #Create a svm Classifier
    svm_clf = svm.SVC(kernel='rbf') # Linear Kernel

    #Train the model using the training sets
    svm_clf.fit(train_data_flatt, train_labels)

    #Predict the response for test dataset
    y_pred_svm = svm_clf.predict(test_data_flatt)

    # --RandomForest--

    rf_clf = RandomForestClassifier()

    rf_clf.fit(train_data_flatt, train_labels)

    y_pred_rf = rf_clf.predict(test_data_flatt)

     # --XGBoost--

    xgb_clf = GradientBoostingClassifier(n_estimators = 100, learning_rate = 1.0, max_depth = 1, random_state = 0)

    xgb_clf.fit(train_data_flatt, train_labels)

    y_pred_xgb = xgb_clf.predict(test_data_flatt)

    print('SVM of sub' + str(sub) + ': ', metrics.accuracy_score(test_labels, y_pred_svm))
    print('RandomForest of sub' + str(sub)+ ': ', metrics.accuracy_score(test_labels, y_pred_rf))
    print('XGBoost of sub' + str(sub) + ': ', metrics.accuracy_score(test_labels, y_pred_xgb))
    print('---------------------------------------------------------------')

In [ ]:
SUBJ_RANGE = range(1,51)

for sub in SUBJ_RANGE:
  if sub == 1 or sub == 2 or sub == 7 or sub == 8 or sub == 16 or sub == 17 or sub == 36 or sub == 42 or sub == 47:
    continue
  else:
    train_path = '/content/drive/MyDrive/Tirocinio/Datasets/SocialMemoryImgs/Wavelet_Abs_Plt_Jet/sub' + str(sub) + '/train'
    test_path = '/content/drive/MyDrive/Tirocinio/Datasets/SocialMemoryImgs/Wavelet_Abs_Plt_Jet/sub' + str(sub) + '/test'
    val_path = '/content/drive/MyDrive/Tirocinio/Datasets/SocialMemoryImgs/Wavelet_Abs_Plt_Jet/sub' + str(sub) + '/val'

    train_transforms = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
    test_transforms = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])
    val_transforms = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()])

    train_dataset, test_dataset, val_dataset = ttv_dataset(train_path, test_path, val_path, train_transforms, test_transforms, val_transforms)

    train_loader, test_loader, val_loader = ttv_loader(train_dataset, test_dataset, val_dataset)
    
    mean, std = mean_and_std(train_loader)

    train_transforms = transforms.Compose([
          transforms.Resize((224,224)),
          transforms.ToTensor(),
          transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))
    ])

    test_transforms = transforms.Compose([
          transforms.Resize((224,224)),
          transforms.ToTensor(),
          transforms.Normalize(torch.Tensor(mean), torch.Tensor(std))    
    ])

    val_transforms = transforms.Compose([
          transforms.Resize((224,224)),
          transforms.ToTensor(),
          transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)) 
    ])


    train_dataset, test_dataset, val_dataset = ttv_dataset(train_path, test_path, val_path, train_transforms, test_transforms, val_transforms)

    train_loader, test_loader, val_loader = ttv_loader(train_dataset, test_dataset, val_dataset)

    train_data_flatt, test_data_flatt, train_labels, test_labels = pretraining(train_loader, test_loader, val_loader)

    ml_classification(train_data_flatt, train_labels, test_data_flatt)




SVM of sub3:  0.7045454545454546
RandomForest of sub3:  0.7045454545454546
XGBoost of sub3:  0.5227272727272727
---------------------------------------------------------------
SVM of sub4:  0.6818181818181818
RandomForest of sub4:  0.5909090909090909
XGBoost of sub4:  0.6818181818181818
---------------------------------------------------------------
SVM of sub5:  0.5
RandomForest of sub5:  0.8181818181818182
XGBoost of sub5:  0.7045454545454546
---------------------------------------------------------------
SVM of sub6:  0.75
RandomForest of sub6:  0.6590909090909091
XGBoost of sub6:  0.5909090909090909
---------------------------------------------------------------
SVM of sub9:  0.6136363636363636
RandomForest of sub9:  0.5909090909090909
XGBoost of sub9:  0.5
---------------------------------------------------------------
SVM of sub10:  0.7045454545454546
RandomForest of sub10:  0.7727272727272727
XGBoost of sub10:  0.7727272727272727
-------------------------------------------------